In [1]:
from numpy import zeros, transpose, asarray, sum,  diag, dot, arccos
from numpy.linalg import norm
import numpy
from scipy.linalg import svd, inv
import matplotlib.pyplot as plt
import re, random, pylab
from math import * 
from operator import itemgetter

In [2]:
# stopwords, retreived from http://www.lextek.com/manuals/onix/stopwords1.html

stopwords = ['a', 'about', 'above', 'across', 'after', 'again', 'against', 
'all', 'almost', 'alone', 'along', 'already', 'also', 'although', 'always', 
'among', 'an', 'and', 'another', 'any', 'anybody', 'anyone', 'anything', 
'anywhere', 'are', 'area', 'areas', 'around', 'as', 'ask', 'asked', 
'asking', 'asks', 'at', 'away', 'b', 'back', 'backed', 'backing', 'backs', 'be', 
'became', 'because', 'become', 'becomes', 'been', 'before', 'began', 'behind', 
'being', 'beings', 'best', 'better', 'between', 'big', 'both', 'but', 'by', 'c', 
'came', 'can', 'cannot', 'case', 'cases', 'certain', 'certainly', 'clear', 'clearly', 
'come', 'could', 'd', 'did', 'differ', 'different', 'differently', 'do', 'does', 'done', 
'down', 'down', 'downed', 'downing', 'downs', 'during', 'e', 'each', 'early', 'either', 
'end', 'ended', 'ending', 'ends', 'enough', 'even', 'evenly', 'ever', 'every', 'everybody', 
'everyone', 'everything', 'everywhere', 'f', 'face', 'faces', 'fact', 'facts', 'far', 
'felt', 'few', 'find', 'finds', 'first', 'for', 'four', 'from', 'full', 'fully', 
'further', 'furthered', 'furthering', 'furthers', 'g', 'gave', 'general', 'generally', 
'get', 'gets', 'give', 'given', 'gives', 'go', 'going', 'good', 'goods', 'got', 'great', 
'greater', 'greatest', 'group', 'grouped', 'grouping', 'groups', 'h', 'had', 'has', 'have', 
'having', 'he', 'her', 'here', 'herself', 'high', 'high', 'high', 'higher', 'highest', 
'him', 'himself', 'his', 'how', 'however', 'i', 'if', 'important', 'in', 'interest', 
'interested', 'interesting', 'interests', 'into', 'is', 'it', 'its', 'itself', 'j', 
'just', 'k', 'keep', 'keeps', 'kind', 'knew', 'know', 'known', 'knows', 'l', 'large', 'largely', 
'last', 'later', 'latest', 'least', 'less', 'let', 'lets', 'like', 'likely', 'long', 'longer',
'longest', 'm', 'made', 'make', 'making', 'man', 'many', 'may', 'me', 'member', 'members', 
'men', 'might', 'more', 'most', 'mostly', 'mr', 'mrs', 'much', 'must', 'my', 'myself', 
'n', 'necessary', 'need', 'needed', 'needing', 'needs', 'never', 'new', 'new', 'newer',
'newest', 'next', 'no', 'nobody', 'non', 'noone', 'not', 'nothing', 'now', 'nowhere', 
'number', 'numbers', 'o', 'of', 'off', 'often', 'old', 'older', 'oldest', 'on',
'once', 'one', 'only', 'open', 'opened', 'opening', 'opens', 'or', 'order', 
'ordered', 'ordering', 'orders', 'other', 'others', 'our', 'out', 'over', 'p', 
'part', 'parted', 'parting', 'parts', 'per', 'perhaps', 'place', 'places', 'point',
'pointed', 'pointing', 'points', 'possible', 'present', 'presented', 'presenting', 
'presents', 'problem', 'problems', 'put', 'puts', 'q', 'quite', 'r', 'rather', 
'really', 'right', 'right', 'room', 'rooms', 's', 'said', 'same', 'saw', 'say', 
'says', 'second', 'seconds', 'see', 'seem', 'seemed', 'seeming', 'seems', 
'sees', 'several', 'shall', 'she', 'should', 'show', 'showed', 'showing', 
'shows', 'side', 'sides', 'since', 'small', 'smaller', 'smallest', 'so', 
'some', 'somebody', 'someone', 'something', 'somewhere', 'state', 'states', 
'still', 'still', 'such', 'sure', 't', 'take', 'taken', 'than', 'that', 'the', 
'their', 'them', 'then', 'there', 'therefore', 'these', 'they', 'thing', 'things', 
'think', 'thinks', 'this', 'those', 'though', 'thought', 'thoughts', 'three', 
'through', 'thus', 'to', 'today', 'together', 'too', 'took', 'toward', 'turn', 
'turned', 'turning', 'turns', 'two', 'u', 'under', 'until', 'up', 'upon', 
'us', 'use', 'used', 'uses', 'v', 'very', 'w', 'want', 'wanted', 'wanting', 
'wants', 'was', 'way', 'ways', 'we', 'well', 'wells', 'went', 'were', 
'what', 'when', 'where', 'whether', 'which', 'while', 'who', 'whole', 'whose', 
'why', 'will', 'with', 'within', 'without', 'work', 'worked', 'working', 'works', 
'would', 'x', 'y', 'year', 'years', 'yet', 'you', 'young', 'younger', 'youngest', 'your', 
'yours', 'z']
ignore_characters = ''',:'!'''

In [3]:
def compare(query1, query2): # core comparison function.
    query1=query1 + " Dummy"
    query2=query2 + " Dummy"
    lsa = LSA(stopwords, ignore_characters)
    queries = [query1,query2]
    for q in queries:
        lsa.parse(q)
    lsa.build()
    lsa.calc()
    Vt = lsa.Vt
    S = diag(lsa.S)
    print(Vt.shape)
    print(S.shape)
    vectors =[(dot(S,Vt[:,0]),dot(S,Vt[:,i])) for i in range(len(Vt))]
    angles = [arccos(dot(a,b)/(norm(a,2)*norm(b,2))) for a,b in vectors[1:]]
    print(len(vectors))
    print(len(angles))
    if len(angles) == 0:
        return 0
    else :
        return str(abs(1 - float(angles[0])/float(pi/2)))

In [10]:
class LSA(object):
    def __init__(self, stopwords, ignore_characters):
        self.stopwords = stopwords
        self.ignore_characters = ignore_characters
        self.wdict = {}
        self.dcount = 0        
    def parse(self, doc):
        words = doc.split();
        for w in words:
            w = w.lower()
            #.translate(None, self.ignore_characters)
            if w in self.stopwords:
                continue
            elif w in self.wdict:
                self.wdict[w].append(self.dcount)
            else:
                self.wdict[w] = [self.dcount]
        self.dcount += 1      
    def build(self): # Create count matrix
        self.keys = [k for k in self.wdict.keys() if len(self.wdict[k]) > 1]
        #if len(self.keys)==0:
         #   self.wdict["Dummy"] = [0]
          #  self.keys.append("Dummy")
        self.keys.sort()
        self.A = zeros([len(self.keys), self.dcount])
        for i, k in enumerate(self.keys):
            for d in self.wdict[k]:
                self.A[i,d] += 1
    def calc(self): # execute SVD
        print(self.A.shape)
        self.U, self.S, self.Vt = svd(self.A, full_matrices =False)
    def TFIDF(self): # calculate tfidf score
        WordsPerDoc = sum(self.A, axis=0)        
        DocsPerWord = sum(asarray(self.A > 0, 'i'), axis=1)
        rows, cols = self.A.shape
        for i in range(rows):
            for j in range(cols):
                self.A[i,j] = (self.A[i,j] / WordsPerDoc[j]) * log(float(cols) / DocsPerWord[i])
    def S(self):
        return self.S
    def U(self):
        return -1 * self.U
    def Vt(self):
        return -1 * self.Vt
    #def printSVD(self):
        #print('Singular values: ')
        #print(self.S)
        #print('U matrix: ')
        #print(-1*self.U[:, 0:3])
        #print('Vt matrix:) '
        #print(-1*self.Vt[0:3, :])

In [11]:
compare('30-year-old Moscow resident was hospitalized with wounds very intimate nature. As it became known LifeNews, in the hands of doctors, the man complained that his casual acquaintance opoila in the sauna, and then gently held his castration operation. And actions criminals were executed with surgical precision - woman sewed all the smallest blood vessels.', 'Mark Zuckerberg')

(1, 2)
(1, 2)
(1, 1)
1
0


0

In [12]:
compare('Hundreds of Palestinians were evacuated from their homes Sunday morning after Israeli authorities opened a number of dams near the border, flooding the Gaza Valley in the wake of a recent severe winter storm.','Hundreds of Palestinians flee floods in Gaza as Israel opens dams')

(5, 2)
(2, 2)
(2, 2)
2
1


'1.0'

In [13]:
compare('Many are expecting Apple talk more about the Apple Watch. However, sources familiar with the matter within Apple have exclusively told The Michael Report that Apple plans to unveil the long-awaited Retina MacBook Air at the same event. The Michael Report has independently verified this information to be highly credible. We should note that this information falls in line with what Apple analysts such as KGI Securities’ Ming-Chi Kuo and Oppenheimer’s Andrew Uerkwitz have been predicting for months. Supply chain sources who spoke to the press have also said that Apple ramped up their production in late 2014 and entered the mass production stage in December, with the goal of producing enough units for an early-2015 debut. Last week, Apple sent out the invites for its “Spring Forward” event, slated to be held at the Yerba Buena Center for the Arts in San Francisco on March 9th.','Woman has surgery to get third breast: The three most statistically unlikely pairs of boobs')

(7, 2)
(2, 2)
(2, 2)
2
1


'0.07627295930784783'

In [22]:
type(compare('Many are expecting Apple talk more about the Apple Watch. However, sources familiar with the matter within Apple have exclusively told The Michael Report that Apple plans to unveil the long-awaited Retina MacBook Air at the same event. The Michael Report has independently verified this information to be highly credible. We should note that this information falls in line with what Apple analysts such as KGI Securities’ Ming-Chi Kuo and Oppenheimer’s Andrew Uerkwitz have been predicting for months. Supply chain sources who spoke to the press have also said that Apple ramped up their production in late 2014 and entered the mass production stage in December, with the goal of producing enough units for an early-2015 debut. Last week, Apple sent out the invites for its “Spring Forward” event, slated to be held at the Yerba Buena Center for the Arts in San Francisco on March 9th.','Woman has surgery to get third breast: The three most statistically unlikely pairs of boobs'))

(7, 2)
(2, 2)
(2, 2)
2
1


str

In [14]:
from csv import DictReader


class DataSet():
    def __init__(self, name="train", path="fnc-1"):
        self.path = path

        print("Reading dataset")
        bodies = name+"_bodies.csv"
        stances = name+"_stances.csv"

        self.stances = self.read(stances)
        articles = self.read(bodies)
        self.articles = dict()

        #make the body ID an integer value
        for s in self.stances:
            s['Body ID'] = int(s['Body ID'])

        #copy all bodies into a dictionary
        for article in articles:
            self.articles[int(article['Body ID'])] = article['articleBody']

        print("Total stances: " + str(len(self.stances)))
        print("Total bodies: " + str(len(self.articles)))



    def read(self,filename):
        rows = []
        with open(self.path + "/" + filename, "r", encoding='utf-8') as table:
            r = DictReader(table)

            for line in r:
                rows.append(line)
        return rows

In [15]:
testD = DataSet(name="competition_test")

Reading dataset
Total stances: 25413
Total bodies: 904


In [20]:
testD.stances[0]

OrderedDict([('Headline',
              'Ferguson riots: Pregnant woman loses eye after cops fire BEAN BAG round through car window'),
             ('Body ID', 2008),
             ('Stance', 'unrelated')])

In [16]:
testD.stances[0]
for stance in testD.stances:
    tempStance = stance['Stance']
    if tempStance == 'disagree':
        stance['Stance'] = 'unrelated'
    elif tempStance != 'unrelated':
        stance['Stance'] = 'related'

In [50]:
%%time
threshold=0.18
headlines_test = []
bodies_test = []
y_out_test = []
test_pred = []
for stance in testD.stances:
    bid = stance['Body ID']
    bodies_test.append(testD.articles[bid])
    headlines_test.append(stance['Headline'])
    y_out_test.append(stance['Stance']=='related')
    if float(compare(stance['Headline'],testD.articles[bid]))>threshold:
        test_pred.append(1)
        print("Threshold = ",threshold,' Iteration = ',i)
        i+=1
    else:
        test_pred.append(0)
        print('Iteration = ',i)
        i+=1
correct = [ i==j for i,j in zip(test_pred,y_out_test)]
t.append(threshold)
a.append(sum(correct)*1.0/len(correct))

(29, 2)
(2, 2)
(2, 2)
2
1
Iteration =  25413
(14, 2)
(2, 2)
(2, 2)
2
1
Iteration =  25414
(19, 2)
(2, 2)
(2, 2)
2
1
Iteration =  25415
(35, 2)
(2, 2)
(2, 2)
2
1
Iteration =  25416
(6, 2)
(2, 2)
(2, 2)
2
1
Iteration =  25417
(6, 2)
(2, 2)
(2, 2)
2
1
Iteration =  25418
(1, 2)
(1, 2)
(1, 1)
1
0
Iteration =  25419
(21, 2)
(2, 2)
(2, 2)
2
1
Iteration =  25420
(93, 2)
(2, 2)
(2, 2)
2
1
Iteration =  25421
(4, 2)
(2, 2)
(2, 2)
2
1
Iteration =  25422
(21, 2)
(2, 2)
(2, 2)
2
1
Threshold =  0.18  Iteration =  25423
(1, 2)
(1, 2)
(1, 1)
1
0
Iteration =  25424
(17, 2)
(2, 2)
(2, 2)
2
1
Threshold =  0.18  Iteration =  25425
(32, 2)
(2, 2)
(2, 2)
2
1
Iteration =  25426
(12, 2)
(2, 2)
(2, 2)
2
1
Threshold =  0.18  Iteration =  25427
(11, 2)
(2, 2)
(2, 2)
2
1
Threshold =  0.18  Iteration =  25428
(9, 2)
(2, 2)
(2, 2)
2
1
Iteration =  25429
(17, 2)
(2, 2)
(2, 2)
2
1
Iteration =  25430
(25, 2)
(2, 2)
(2, 2)
2
1
Iteration =  25431
(24, 2)
(2, 2)
(2, 2)
2
1
Iteration =  25432
(10, 2)
(2, 2)
(2, 2)
2
1
Thre

(2, 2)
2
1
Iteration =  25722
(17, 2)
(2, 2)
(2, 2)
2
1
Threshold =  0.18  Iteration =  25723
(18, 2)
(2, 2)
(2, 2)
2
1
Iteration =  25724
(37, 2)
(2, 2)
(2, 2)
2
1
Iteration =  25725
(41, 2)
(2, 2)
(2, 2)
2
1
Iteration =  25726
(22, 2)
(2, 2)
(2, 2)
2
1
Iteration =  25727
(34, 2)
(2, 2)
(2, 2)
2
1
Iteration =  25728
(3, 2)
(2, 2)
(2, 2)
2
1
Threshold =  0.18  Iteration =  25729
(2, 2)
(2, 2)
(2, 2)
2
1
Threshold =  0.18  Iteration =  25730
(25, 2)
(2, 2)
(2, 2)
2
1
Threshold =  0.18  Iteration =  25731
(17, 2)
(2, 2)
(2, 2)
2
1
Threshold =  0.18  Iteration =  25732
(25, 2)
(2, 2)
(2, 2)
2
1
Iteration =  25733
(15, 2)
(2, 2)
(2, 2)
2
1
Threshold =  0.18  Iteration =  25734
(13, 2)
(2, 2)
(2, 2)
2
1
Iteration =  25735
(74, 2)
(2, 2)
(2, 2)
2
1
Iteration =  25736
(17, 2)
(2, 2)
(2, 2)
2
1
Iteration =  25737
(19, 2)
(2, 2)
(2, 2)
2
1
Iteration =  25738
(22, 2)
(2, 2)
(2, 2)
2
1
Iteration =  25739
(19, 2)
(2, 2)
(2, 2)
2
1
Iteration =  25740
(1, 2)
(1, 2)
(1, 1)
1
0
Iteration =  25741
(13,

(2, 2)
2
1
Iteration =  26032
(9, 2)
(2, 2)
(2, 2)
2
1
Iteration =  26033
(11, 2)
(2, 2)
(2, 2)
2
1
Threshold =  0.18  Iteration =  26034
(44, 2)
(2, 2)
(2, 2)
2
1
Iteration =  26035
(11, 2)
(2, 2)
(2, 2)
2
1
Iteration =  26036
(15, 2)
(2, 2)
(2, 2)
2
1
Iteration =  26037
(12, 2)
(2, 2)
(2, 2)
2
1
Iteration =  26038
(13, 2)
(2, 2)
(2, 2)
2
1
Iteration =  26039
(8, 2)
(2, 2)
(2, 2)
2
1
Iteration =  26040
(26, 2)
(2, 2)
(2, 2)
2
1
Iteration =  26041
(12, 2)
(2, 2)
(2, 2)
2
1
Iteration =  26042
(15, 2)
(2, 2)
(2, 2)
2
1
Threshold =  0.18  Iteration =  26043
(34, 2)
(2, 2)
(2, 2)
2
1
Iteration =  26044
(17, 2)
(2, 2)
(2, 2)
2
1
Iteration =  26045
(39, 2)
(2, 2)
(2, 2)
2
1
Threshold =  0.18  Iteration =  26046
(27, 2)
(2, 2)
(2, 2)
2
1
Iteration =  26047
(25, 2)
(2, 2)
(2, 2)
2
1
Iteration =  26048
(13, 2)
(2, 2)
(2, 2)
2
1
Iteration =  26049
(10, 2)
(2, 2)
(2, 2)
2
1
Iteration =  26050
(6, 2)
(2, 2)
(2, 2)
2
1
Iteration =  26051
(59, 2)
(2, 2)
(2, 2)
2
1
Iteration =  26052
(8, 2)
(2, 2)
(2

(128, 2)
(2, 2)
(2, 2)
2
1
Iteration =  26326
(21, 2)
(2, 2)
(2, 2)
2
1
Iteration =  26327
(1, 2)
(1, 2)
(1, 1)
1
0
Iteration =  26328
(11, 2)
(2, 2)
(2, 2)
2
1
Iteration =  26329
(25, 2)
(2, 2)
(2, 2)
2
1
Iteration =  26330
(43, 2)
(2, 2)
(2, 2)
2
1
Iteration =  26331
(23, 2)
(2, 2)
(2, 2)
2
1
Threshold =  0.18  Iteration =  26332
(17, 2)
(2, 2)
(2, 2)
2
1
Iteration =  26333
(10, 2)
(2, 2)
(2, 2)
2
1
Iteration =  26334
(2, 2)
(2, 2)
(2, 2)
2
1
Threshold =  0.18  Iteration =  26335
(132, 2)
(2, 2)
(2, 2)
2
1
Iteration =  26336
(28, 2)
(2, 2)
(2, 2)
2
1
Iteration =  26337
(34, 2)
(2, 2)
(2, 2)
2
1
Iteration =  26338
(23, 2)
(2, 2)
(2, 2)
2
1
Iteration =  26339
(17, 2)
(2, 2)
(2, 2)
2
1
Iteration =  26340
(27, 2)
(2, 2)
(2, 2)
2
1
Iteration =  26341
(19, 2)
(2, 2)
(2, 2)
2
1
Threshold =  0.18  Iteration =  26342
(15, 2)
(2, 2)
(2, 2)
2
1
Iteration =  26343
(18, 2)
(2, 2)
(2, 2)
2
1
Iteration =  26344
(9, 2)
(2, 2)
(2, 2)
2
1
Iteration =  26345
(35, 2)
(2, 2)
(2, 2)
2
1
Threshold =  0.18 

1
Iteration =  26515
(22, 2)
(2, 2)
(2, 2)
2
1
Threshold =  0.18  Iteration =  26516
(35, 2)
(2, 2)
(2, 2)
2
1
Iteration =  26517
(18, 2)
(2, 2)
(2, 2)
2
1
Iteration =  26518
(24, 2)
(2, 2)
(2, 2)
2
1
Iteration =  26519
(17, 2)
(2, 2)
(2, 2)
2
1
Iteration =  26520
(6, 2)
(2, 2)
(2, 2)
2
1
Threshold =  0.18  Iteration =  26521
(103, 2)
(2, 2)
(2, 2)
2
1
Iteration =  26522
(24, 2)
(2, 2)
(2, 2)
2
1
Iteration =  26523
(11, 2)
(2, 2)
(2, 2)
2
1
Iteration =  26524
(62, 2)
(2, 2)
(2, 2)
2
1
Iteration =  26525
(51, 2)
(2, 2)
(2, 2)
2
1
Iteration =  26526
(10, 2)
(2, 2)
(2, 2)
2
1
Iteration =  26527
(23, 2)
(2, 2)
(2, 2)
2
1
Iteration =  26528
(16, 2)
(2, 2)
(2, 2)
2
1
Threshold =  0.18  Iteration =  26529
(29, 2)
(2, 2)
(2, 2)
2
1
Threshold =  0.18  Iteration =  26530
(2, 2)
(2, 2)
(2, 2)
2
1
Threshold =  0.18  Iteration =  26531
(14, 2)
(2, 2)
(2, 2)
2
1
Iteration =  26532
(103, 2)
(2, 2)
(2, 2)
2
1
Iteration =  26533
(3, 2)
(2, 2)
(2, 2)
2
1
Threshold =  0.18  Iteration =  26534
(7, 2)
(2, 

...

Threshold =  0.18  Iteration =  49594
(2, 2)
(2, 2)
(2, 2)
2
1
Threshold =  0.18  Iteration =  49595
(1, 2)
(1, 2)
(1, 1)
1
0
Iteration =  49596
(6, 2)
(2, 2)
(2, 2)
2
1
Iteration =  49597
(4, 2)
(2, 2)
(2, 2)
2
1
Iteration =  49598
(64, 2)
(2, 2)
(2, 2)
2
1
Iteration =  49599
(11, 2)
(2, 2)
(2, 2)
2
1
Iteration =  49600
(13, 2)
(2, 2)
(2, 2)
2
1
Iteration =  49601
(13, 2)
(2, 2)
(2, 2)
2
1
Iteration =  49602
(25, 2)
(2, 2)
(2, 2)
2
1
Iteration =  49603
(9, 2)
(2, 2)
(2, 2)
2
1
Iteration =  49604
(17, 2)
(2, 2)
(2, 2)
2
1
Iteration =  49605
(2, 2)
(2, 2)
(2, 2)
2
1
Threshold =  0.18  Iteration =  49606
(93, 2)
(2, 2)
(2, 2)
2
1
Threshold =  0.18  Iteration =  49607
(18, 2)
(2, 2)
(2, 2)
2
1
Iteration =  49608
(28, 2)
(2, 2)
(2, 2)
2
1
Threshold =  0.18  Iteration =  49609
(22, 2)
(2, 2)
(2, 2)
2
1
Iteration =  49610
(11, 2)
(2, 2)
(2, 2)
2
1
Iteration =  49611
(7, 2)
(2, 2)
(2, 2)
2
1
Threshold =  0.18  Iteration =  49612
(20, 2)
(2, 2)
(2, 2)
2
1
Iteration =  49613
(11, 2)
(2, 2)
(2, 

1
Threshold =  0.18  Iteration =  49789
(3, 2)
(2, 2)
(2, 2)
2
1
Threshold =  0.18  Iteration =  49790
(18, 2)
(2, 2)
(2, 2)
2
1
Threshold =  0.18  Iteration =  49791
(135, 2)
(2, 2)
(2, 2)
2
1
Iteration =  49792
(3, 2)
(2, 2)
(2, 2)
2
1
Threshold =  0.18  Iteration =  49793
(14, 2)
(2, 2)
(2, 2)
2
1
Iteration =  49794
(34, 2)
(2, 2)
(2, 2)
2
1
Iteration =  49795
(23, 2)
(2, 2)
(2, 2)
2
1
Iteration =  49796
(18, 2)
(2, 2)
(2, 2)
2
1
Iteration =  49797
(45, 2)
(2, 2)
(2, 2)
2
1
Iteration =  49798
(19, 2)
(2, 2)
(2, 2)
2
1
Iteration =  49799
(37, 2)
(2, 2)
(2, 2)
2
1
Iteration =  49800
(45, 2)
(2, 2)
(2, 2)
2
1
Iteration =  49801
(14, 2)
(2, 2)
(2, 2)
2
1
Threshold =  0.18  Iteration =  49802
(10, 2)
(2, 2)
(2, 2)
2
1
Threshold =  0.18  Iteration =  49803
(12, 2)
(2, 2)
(2, 2)
2
1
Iteration =  49804
(32, 2)
(2, 2)
(2, 2)
2
1
Iteration =  49805
(21, 2)
(2, 2)
(2, 2)
2
1
Iteration =  49806
(15, 2)
(2, 2)
(2, 2)
2
1
Iteration =  49807
(36, 2)
(2, 2)
(2, 2)
2
1
Iteration =  49808
(13, 2)
(2,

2
1
Iteration =  50110
(9, 2)
(2, 2)
(2, 2)
2
1
Iteration =  50111
(13, 2)
(2, 2)
(2, 2)
2
1
Iteration =  50112
(13, 2)
(2, 2)
(2, 2)
2
1
Iteration =  50113
(27, 2)
(2, 2)
(2, 2)
2
1
Iteration =  50114
(16, 2)
(2, 2)
(2, 2)
2
1
Iteration =  50115
(25, 2)
(2, 2)
(2, 2)
2
1
Iteration =  50116
(7, 2)
(2, 2)
(2, 2)
2
1
Iteration =  50117
(3, 2)
(2, 2)
(2, 2)
2
1
Threshold =  0.18  Iteration =  50118
(8, 2)
(2, 2)
(2, 2)
2
1
Threshold =  0.18  Iteration =  50119
(14, 2)
(2, 2)
(2, 2)
2
1
Threshold =  0.18  Iteration =  50120
(362, 2)
(2, 2)
(2, 2)
2
1
Iteration =  50121
(5, 2)
(2, 2)
(2, 2)
2
1
Threshold =  0.18  Iteration =  50122
(7, 2)
(2, 2)
(2, 2)
2
1
Threshold =  0.18  Iteration =  50123
(45, 2)
(2, 2)
(2, 2)
2
1
Iteration =  50124
(7, 2)
(2, 2)
(2, 2)
2
1
Iteration =  50125
(96, 2)
(2, 2)
(2, 2)
2
1
Iteration =  50126
(1, 2)
(1, 2)
(1, 1)
1
0
Iteration =  50127
(12, 2)
(2, 2)
(2, 2)
2
1
Iteration =  50128
(52, 2)
(2, 2)
(2, 2)
2
1
Iteration =  50129
(8, 2)
(2, 2)
(2, 2)
2
1
Iteration

(2, 2)
2
1
Threshold =  0.18  Iteration =  50390
(19, 2)
(2, 2)
(2, 2)
2
1
Iteration =  50391
(46, 2)
(2, 2)
(2, 2)
2
1
Iteration =  50392
(100, 2)
(2, 2)
(2, 2)
2
1
Iteration =  50393
(31, 2)
(2, 2)
(2, 2)
2
1
Iteration =  50394
(43, 2)
(2, 2)
(2, 2)
2
1
Threshold =  0.18  Iteration =  50395
(4, 2)
(2, 2)
(2, 2)
2
1
Iteration =  50396
(12, 2)
(2, 2)
(2, 2)
2
1
Iteration =  50397
(36, 2)
(2, 2)
(2, 2)
2
1
Threshold =  0.18  Iteration =  50398
(13, 2)
(2, 2)
(2, 2)
2
1
Iteration =  50399
(7, 2)
(2, 2)
(2, 2)
2
1
Iteration =  50400
(13, 2)
(2, 2)
(2, 2)
2
1
Iteration =  50401
(34, 2)
(2, 2)
(2, 2)
2
1
Iteration =  50402
(32, 2)
(2, 2)
(2, 2)
2
1
Iteration =  50403
(41, 2)
(2, 2)
(2, 2)
2
1
Iteration =  50404
(21, 2)
(2, 2)
(2, 2)
2
1
Iteration =  50405
(2, 2)
(2, 2)
(2, 2)
2
1
Threshold =  0.18  Iteration =  50406
(22, 2)
(2, 2)
(2, 2)
2
1
Threshold =  0.18  Iteration =  50407
(1, 2)
(1, 2)
(1, 1)
1
0
Iteration =  50408
(13, 2)
(2, 2)
(2, 2)
2
1
Iteration =  50409
(2, 2)
(2, 2)
(2, 2)
2


(2, 2)
(2, 2)
2
1
Threshold =  0.18  Iteration =  50644
(50, 2)
(2, 2)
(2, 2)
2
1
Threshold =  0.18  Iteration =  50645
(146, 2)
(2, 2)
(2, 2)
2
1
Threshold =  0.18  Iteration =  50646
(193, 2)
(2, 2)
(2, 2)
2
1
Threshold =  0.18  Iteration =  50647
(69, 2)
(2, 2)
(2, 2)
2
1
Threshold =  0.18  Iteration =  50648
(71, 2)
(2, 2)
(2, 2)
2
1
Threshold =  0.18  Iteration =  50649
(50, 2)
(2, 2)
(2, 2)
2
1
Threshold =  0.18  Iteration =  50650
(145, 2)
(2, 2)
(2, 2)
2
1
Threshold =  0.18  Iteration =  50651
(193, 2)
(2, 2)
(2, 2)
2
1
Threshold =  0.18  Iteration =  50652
(68, 2)
(2, 2)
(2, 2)
2
1
Threshold =  0.18  Iteration =  50653
(71, 2)
(2, 2)
(2, 2)
2
1
Threshold =  0.18  Iteration =  50654
(50, 2)
(2, 2)
(2, 2)
2
1
Threshold =  0.18  Iteration =  50655
(145, 2)
(2, 2)
(2, 2)
2
1
Threshold =  0.18  Iteration =  50656
(193, 2)
(2, 2)
(2, 2)
2
1
Threshold =  0.18  Iteration =  50657
(68, 2)
(2, 2)
(2, 2)
2
1
Threshold =  0.18  Iteration =  50658
(72, 2)
(2, 2)
(2, 2)
2
1
Threshold =  0.1

In [51]:
correct = [ i==j for i,j in zip(test_pred,y_out_test)]
sum(correct)*1.0/len(correct)

0.87687404084523668